In [ ]:
!pip install haversine

In [ ]:
import keys
import pandas as pd
import requests
from pprint import pprint
import json
import numpy as np

In [ ]:
screens = pd.read_csv('screen geocoded.csv')
screens.drop('Unnamed: 0', axis=1, inplace=True)
screens.shape

In [ ]:
screens.head()

In [ ]:
# look at number that are missing lats and longs
print(screens[screens.lat.isnull() == True].shape)
print(screens[screens.lat.isnull() == False].shape)
print()

In [ ]:
########################################

In [ ]:
# add close date from model-data file
model_data = pd.read_csv('model_data.csv')

In [ ]:
model_data.head()

In [ ]:
close_dates = model_data[['name', 'close_date'
                          #,'address'
                          ]]

In [ ]:
close_dates.shape

In [ ]:
screens = screens.merge(close_dates, how='left', right_on='name', left_on='Opportunity: Opportunity Name'
                        #,indicator=True
                        )

In [ ]:
# why did the number of records go from 628 to 649? are there dups?
print(screens.shape)
print(screens[screens.close_date.isnull() == False].shape)
print(screens[screens.close_date.isnull() == True].shape)

In [ ]:
screens.drop(['name'], axis=1, inplace=True)

In [ ]:
screens[screens.close_date.isnull() == True].head(100)

unmatching screen addresses:
WeDriveU-AthenaHealth - Arsenal Campus- Revisit

Post Alexander - ATL

Matrix Residential - ATL - Local on 14th

Seven Oaks - ATL - 3003 Perimeter Summit



In [ ]:
model_data[model_data.lat == 33.772828]

It looks like this screen didn't exist in close dates

In [ ]:
model_data[model_data.stage == 'Closed Won'].shape

In [ ]:
screens.drop_duplicates(['lat'], keep='first', inplace=True)

In [ ]:
screens.to_csv('screens_and_close_dates.csv', index=False)

In [ ]:
################################################

In [ ]:
from haversine import haversine

In [ ]:
def find_closest(point, list_of_points, debug=False):
    distances = []
    for i in list_of_points:
        # if it is a different point, calculate the distance
        if point != i:
            dist = haversine(point, i)
            if debug:
                print(point, i, dist)
            distances.append(dist)
    # return the smallest distance
    # do we want to set a max here?
    return (distances[np.argmin(distances)])

In [ ]:
point = (38.876059, -77.020240)
list_of_points = [(40.754448, -74.028046), (38.999289, -
                                            77.028885), (38.876059, -77.020240)]
find_closest(point, list_of_points, True)

In [ ]:
point = (38.876059, -77.020240)
list_of_points = [(40.754448, -74.028046), (38.999289, -77.028885)]
find_closest(point, list_of_points)

In [ ]:
def find_within_distance(point, list_of_points, radius=.5):
    count = 0
    for i in list_of_points:
        dist = haversine(point, i)
        # if point == i:
        #    print('found self', i)
        if point != i:
            if dist <= radius:
                count += 1
    return (count)

In [ ]:
point = (38.876059, -77.020240)
list_of_points = [(40.754448, -74.028046), (38.999289, -77.028885)]
find_within_distance(point, list_of_points, 14)

In [ ]:
model_data.shape

In [ ]:
print(screens.shape)
print(screens[screens.close_date.isnull() == False].shape)
print(screens[screens.close_date.isnull() == True].shape)

In [ ]:
# build a list of all points we want to checks
scr = screens[screens.lng.isnull() == False][['lat', 'lng']]

In [ ]:
screen_list = list(zip(scr['lat'], scr['lng']))

In [ ]:
find_closest((38.876059, -77.020240), screen_list)

In [ ]:
screens[screens['Screen Addresss (City)'] == 'Silver Spring']

In [ ]:
find_within_distance(
    (33.782524200000005, -84.380598800000001), screen_list, radius=1000000)

In [ ]:
model_data['lat_lng'] = list(zip(model_data['lat'], model_data['lng']))

In [ ]:
model_data['closest_ts'] = np.nan

In [ ]:
for i, row in model_data.iterrows():
    closest_ts = find_closest(row.lat_lng, screen_list)
    model_data.loc[model_data['id'] == row.id, 'closest_ts'] = closest_ts

In [ ]:
model_data.sort_values('closest_ts', ascending=False)

In [ ]:
model_data['within_one_tenth'] = np.nan
model_data['within_one_half'] = np.nan
model_data['within_one'] = np.nan
model_data['within_five'] = np.nan

In [ ]:
for i, row in model_data.iterrows():
    within_one_tenth = find_within_distance(row.lat_lng, screen_list, .1)
    within_one_half = find_within_distance(row.lat_lng, screen_list, .5)
    within_one = find_within_distance(row.lat_lng, screen_list, 1)
    within_five = find_within_distance(row.lat_lng, screen_list, 5)
    model_data.loc[model_data['id'] == row.id,
                   'within_one_tenth'] = within_one_tenth
    model_data.loc[model_data['id'] == row.id,
                   'within_one_half'] = within_one_half
    model_data.loc[model_data['id'] == row.id, 'within_one'] = within_one
    model_data.loc[model_data['id'] == row.id, 'within_five'] = within_five

In [ ]:
model_data.head(100)

In [ ]:
df.to_csv('model_data.csv', index=False)